In [38]:
from dataclasses import dataclass
from typing import Annotated, Any
from typing_extensions import TypeAliasType

from pydantic import BeforeValidator, AfterValidator, WrapValidator, PlainValidator, BaseModel, PlainSerializer, WithJsonSchema, GetJsonSchemaHandler, Field

from pydantic_core import core_schema

In [39]:
@dataclass
class ThirdParty:
    value: Any

In [40]:
def my_validator(value, info):
    print(f"{info=}")
    return ThirdParty(value)

In [41]:
class MyValidator:
    @classmethod
    def __get_pydantic_core_schema__(cls, source_type, handler):
        def simple_validate(value, info):
            print(f"{value=}")
            print(f"{info=}")
            print(f"{', '.join(h for h in dir(handler) if not h.startswith('_'))}")
            print(f"{handler.field_name=}")
            return ThirdParty(value)

        return core_schema.with_info_plain_validator_function(simple_validate, field_name=handler.field_name)

    # @classmethod
    # def __get_pydantic_json_schema__(
    #     cls, core_schema: core_schema.CoreSchema, handler: GetJsonSchemaHandler
    # ):
    #     print(f"{dir(handler)=}")
    #     return core_schema
        

In [42]:
ThirdPartyType = TypeAliasType(
    'ThirdPartyType',
    Annotated[
        ThirdParty,
        PlainValidator(my_validator),
        WithJsonSchema(core_schema.int_schema())
    ]
)

In [43]:
class MyModel(BaseModel):
    one: ThirdPartyType
    two: ThirdPartyType
    three: Annotated[ThirdPartyType, Field(title="three")]

In [46]:
mod = MyModel(one=1, two=2, three=3)

info=ValidationInfo(config={'title': 'MyModel'}, context=None, data={}, field_name='one')
info=ValidationInfo(config={'title': 'MyModel'}, context=None, data={'one': ThirdParty(value=1)}, field_name='one')
info=ValidationInfo(config={'title': 'MyModel'}, context=None, data={'one': ThirdParty(value=1), 'two': ThirdParty(value=2)}, field_name='one')


In [47]:
mod

MyModel(one=ThirdParty(value=1), two=ThirdParty(value=2), three=ThirdParty(value=3))

In [48]:
mod.model_dump()

{'one': {'value': 1}, 'two': {'value': 2}, 'three': {'value': 3}}

In [49]:
mod.model_dump_json()

'{"one":{"value":1},"two":{"value":2},"three":{"value":3}}'

In [50]:
mod.model_json_schema()

{'$defs': {'ThirdPartyType': {'type': 'int'}},
 'properties': {'one': {'$ref': '#/$defs/ThirdPartyType'},
  'two': {'$ref': '#/$defs/ThirdPartyType'},
  'three': {'allOf': [{'$ref': '#/$defs/ThirdPartyType'}], 'title': 'three'}},
 'required': ['one', 'two', 'three'],
 'title': 'MyModel',
 'type': 'object'}

In [54]:
class IntModel(BaseModel):
    one: int
    two: int

In [55]:
my_int = IntModel(one=1, two=2)

In [56]:
my_int.model_json_schema()

{'properties': {'one': {'title': 'One', 'type': 'integer'},
  'two': {'title': 'Two', 'type': 'integer'}},
 'required': ['one', 'two'],
 'title': 'IntModel',
 'type': 'object'}

In [6]:
MyType = Annotated[
        MyClass,
        MyValidator(),
    ]

    

In [11]:
class MyModel(BaseModel):
    # one: MyType
    two: MyType

In [12]:
mod = MyModel(one=6, two=5)

value=5
info=ValidationInfo(config={'title': 'MyModel'}, context=None, data={}, field_name='two')
field_name, generate_schema, resolve_ref_schema
handler.field_name=None


In [13]:
mod

MyModel(two=MyClass(value=5))

In [14]:
mod.model_json_schema()

PydanticInvalidForJsonSchema: Cannot generate a JsonSchema for core_schema.PlainValidatorFunctionSchema ({'type': 'with-info', 'function': <function MyValidator.__get_pydantic_core_schema__.<locals>.simple_validate at 0x10b780c20>, 'field_name': 'two'})

For further information visit https://errors.pydantic.dev/2.5/u/invalid-for-json-schema